In [2]:
import pandas as pd
import numpy as np
import subprocess
import json
import seaborn as sns
import os

In [3]:
import_df = pd.read_json('zot_crossref_processed.json', 
                                orient='records')

df = import_df[import_df['fail'] == True][import_df['DOI'] != '']

rest_df = import_df.loc[import_df.index.difference(df.index)]
bad_df = import_df[import_df['fail'] == True][import_df['DOI'] == '']
good_df = import_df[import_df['fail'] == False]

/var/folders/pk/zld5ctsn7r9cvw4wpxfg6g8w0000gn/T/ipykernel_44881/4076266310.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = import_df[import_df['fail'] == True][import_df['DOI'] != '']
/var/folders/pk/zld5ctsn7r9cvw4wpxfg6g8w0000gn/T/ipykernel_44881/4076266310.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bad_df = import_df[import_df['fail'] == True][import_df['DOI'] == '']


In [5]:
len(import_df)

4675

In [20]:
zjson_df, bib_df = pd.DataFrame(), pd.DataFrame()

counter = 0
repeat_run = True
prev_url = ""
failed, perffail = [], []
for index, row in df.iterrows():
        
    # call bash script
    rc = subprocess.call("./zot2.sh '%s'" % (row['DOI']), shell=True)
    # create df for this url
    perf = False
    
                
    try:
        while True:
            temp1_df = pd.read_json('temp1.json', orient='records')
            # manual error handling
            if ("url" in temp1_df) and temp1_df["url"].str.contains("validate.perfdrive")[0]:
                perf = True
                break
            url_exists = "url" in temp1_df
            with open("temp2.json") as f:
                temp1_df["bib"] = f.read()
            if not url_exists:
                break
            if (temp1_df["url"] != prev_url)[0]:
                break
        if not perf:
            if url_exists:
                prev_url = temp1_df["url"]

            temp1_df['tURL'] = row['tURL']
            temp1_df["index"] = row['index']
            temp1_df['DOI'] = row['DOI']
            temp1_df['fail'] = False
            temp1_df['perf'] = False
            if "ShortNames" in row:
                temp1_df['ShortNames'] = [row['ShortNames']]

            if "date" in temp1_df:
                temp1_df["year"] = temp1_df['date'].astype(str).apply(lambda x: '(' + x[0:4] + ')' if x[0] != 'n' else x)
            
            if "creators" in temp1_df:
                splitlist = str(temp1_df["creators"][0]).split('\'')
                temp1_df["author"] = splitlist[3][0] + ' ' + splitlist[7]
            
            '''
            if ('extra' in temp1_df):
                extra_str = temp1_df["extra"][0]
                if (type(extra_str) == str) and extra_str[0:3] == 'DOI':
                    print("doi from extra: " + str('DOI' in temp1_df) + " " + extra_str[5:])
                    temp1_df["DOI"] = extra_str[5:]
            '''

            zjson_df = pd.concat([zjson_df, temp1_df], sort=False)
            print(counter)
        else:
            perffail.append(counter)
            temp_df = pd.DataFrame({'fail': [True], 'tURL': [row['tURL']], 'perf': [True], 'index': [row['index']]})
            zjson_df = pd.concat([zjson_df, temp_df], sort=False)
            print('Perfdrive: ' + str(counter))
        
    except:
        failed.append(counter)
        temp_df = pd.DataFrame({'fail': [True], 'tURL': [row['tURL']], 'perf': [False], 'index': [row['index']]})
        zjson_df = pd.concat([zjson_df, temp_df], sort=False)
        print('Failed: ' + str(counter))
    
    counter += 1
    
    
    
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   743  100   713  100    30    490     20  0:00:01  0:00:01 --:--:--   513
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1229  100   516  100   713  24711  34145 --:--:-- --:--:-- --:--:--  120k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

0


100  2108  100  2077  100    31   2192     32 --:--:-- --:--:-- --:--:--  2237
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3721  100  1710  100  2011  95434   109k --:--:-- --:--:-- --:--:--  726k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

1


100  1156  100  1139  100    17   1252     18 --:--:-- --:--:-- --:--:--  1278
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1721  100   582  100  1139  25252  49420 --:--:-- --:--:-- --:--:--  186k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

...

100   954  100   924  100    30   1031     33 --:--:-- --:--:-- --:--:--  1070
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1553  100   629  100   924  28111  41296 --:--:-- --:--:-- --:--:--  189k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

490


100   959  100   934  100    25    927     24  0:00:01  0:00:01 --:--:--   959
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   944  100    11  100   933    682  57910 --:--:-- --:--:-- --:--:--  230k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

Failed: 491
492


100  2054  100  2023  100    31   1841     28  0:00:01  0:00:01 --:--:--  1884
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3870  100  1862  100  2008  91225  98378 --:--:-- --:--:-- --:--:--  472k


In [21]:
final_df = pd.concat([good_df, zjson_df], sort=False)

In [22]:
final_df.to_csv('zot2.csv')